In [1]:
# Import standard libraries
import os
from contextlib import redirect_stdout

import sys
# append coeqwal packages to path
sys.path.append('./coeqwalpackage')

import numpy as np
import pandas as pd
import datetime as dt


In [2]:
# Import custom libraries
# Note: on my computer the next import doesn't work the first time I call it, why? If I re-run the cell, then it is ok. MUST DEBUG
from coeqwalpackage.metrics import *
import cqwlutils as cu
import plotting as pu

## Define contol file name

In [4]:
CtrlFile = 'CalSim3DataExtractionInitFile_v3.xlsx'
CtrlTab = 'Init'

## Read from control file

In [6]:
ScenarioListFile, ScenarioListTab, ScenarioListPath, DVDssNamesOutPath, SVDssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, DVDssMin, DVDssMax, SVDssMin, SVDssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax = cu.read_init_file(CtrlFile, CtrlTab)

### Read in Data

In [8]:
df, dss_names = read_in_df(ConvertDataOutPath,DVDssNamesOutPath)

In [9]:
df = add_water_year_column(df)

C:\Users\isabe\COEQWAL\coeqwal\notebooks\coeqwalpackage\metrics.py:175: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  return df_copy.drop(["Date", "Year", "Month"], axis=1)


### Get metrics paths and make directories if needed

In [11]:
metrics_path = GroupDataDirPath + "/metrics_output"
if not os.path.exists(metrics_path):
    os.makedirs(metrics_path)

## Metrics Examples

In [13]:
# To get a single metric and variable dataframe, type metrics[Month_{var}_{metric}] ie metrics['Apr_S_RESTOT_NOD_mnth_avg'], thresholds["S_SHSTA_frequency_hitting_level5"]
# Month = Apr, Sept... or None
# Metric = [ann_avg, mnth_avg, compute_iqr, ann_percentile, moy_avgs, mnth_percentile, annual_totals]

In [14]:
# Thresholds

variables = ["S_SHSTA_", "S_OROVL_", "S_TRNTY_", "S_SLUIS_CVP_", "S_SLUIS_SWP_"]
variables_level1 = ["S_SHSTALEVEL1DV", "S_OROVLLEVEL1DV", "S_TRNTYLEVEL1DV", "S_SLUIS_CVPLEVEL1DV", "S_SLUIS_SWPLEVEL1DV"]
variables_level5 = ["S_SHSTALEVEL5DV", "S_OROVLLEVEL5DV", "S_TRNTYLEVEL5DV", "S_SLUIS_CVPLEVEL5DV", "S_SLUIS_SWPLEVEL5DV"]

In [15]:
thresholds = {}

for var, var_lvl1, var_lvl5 in zip(variables, variables_level1, variables_level5):
    # LEVEL5DV calculation (LEVEL5DV - var < 0)
    exceedance_days, exceedance_days_percent = frequency_hitting_level(df, var, var_lvl5, "TAF", f"All_Prob_{var}flood")
    thresholds[f"{var}frequency_hitting_level5"] = (exceedance_days, exceedance_days_percent)

    # LEVEL1DV calculation (var - LEVEL1DV < 0)
    exceedance_days, exceedance_days_percent = frequency_hitting_level(df, var_lvl1, var, "TAF", f"All_Prob_{var}dead")
    thresholds[f"{var}frequency_hitting_level1"] = (exceedance_days, exceedance_days_percent)

    # September LEVEL1DV calculation
    exceedance_days, exceedance_days_percent = frequency_hitting_level(df, var_lvl1, var, "TAF", f"Sept_Prob_{var}dead", months=[9])
    thresholds[f"Sept_{var}frequency_hitting_level1"] = (exceedance_days, exceedance_days_percent)

for threshold, (threshold_df, threshold_percent) in thresholds.items():
    print(f"Threshold: {threshold}")
    display(threshold_df)
    display(threshold_percent)

Threshold: S_SHSTA_frequency_hitting_level5


,All_Prob_S_SHSTA_flood
0,400.0
1,378.0
2,267.0
3,231.0
4,211.0
5,364.0
6,362.0
7,361.0
8,361.0
9,362.0


A            CALSIM                                                          \
B     S_SHSTA_s0001 S_SHSTA_s0002 S_SHSTA_s0003 S_SHSTA_s0004 S_SHSTA_s0005   
C           STORAGE       STORAGE       STORAGE       STORAGE       STORAGE   
D              1MON          1MON          1MON          1MON          1MON   
E            L2020A        L2020A        L2020A        L2020A        L2020A   
F          PER-AVER      PER-AVER      PER-AVER      PER-AVER      PER-AVER   
Units           TAF           TAF           TAF           TAF           TAF   
0         33.333333          31.5         22.25         19.25     17.583333   

A                                                                            \
B     S_SHSTA_s0006 S_SHSTA_s0007 S_SHSTA_s0008 S_SHSTA_s0009 S_SHSTA_s0010   
C           STORAGE       STORAGE       STORAGE       STORAGE       STORAGE   
D              1MON          1MON          1MON          1MON          1MON   
E            L2020A        L2020A        L2020A        L2020A        L2020A   
F          PER-AVER      PER-AVER      PER-AVER      PER-AVER      PER-AVER   
Units           TAF           TAF           TAF           TAF           TAF   
0         30.333333     30.166667     30.083333     30.083333     30.166667   

A                                                                            \
B     S_SHSTA_s0011 S_SHSTA_s0012 S_SHSTA_s0013 S_SHSTA_s0014 S_SHSTA_s0015   
C           STORAGE       STORAGE       STORAGE       STORAGE       STORAGE   
D              1MON          1MON          1MON          1MON          1MON   
E            L2020A        L2020A        L2020A        L2020A        L2020A   
F          PER-AVER      PER-AVER      PER-AVER      PER-AVER      PER-AVER   
Units           TAF           TAF           TAF           TAF           TAF   
0         31.583333          33.0     19.666667     20.583333     32.916667   

A                    
B     S_SHSTA_s0016  
C           STORAGE  
D              1MON  
E            L2020A  
F          PER-AVER  
Units           TAF  
0         20.833333

Threshold: S_SHSTA_frequency_hitting_level1


,All_Prob_S_SHSTA_dead
0,5.0
1,21.0
2,46.0
3,63.0
4,64.0
5,4.0
6,0.0
7,0.0
8,0.0
9,0.0


...

,All_Prob_S_SLUIS_SWP_dead
0,35.0
1,32.0
2,33.0
3,67.0
4,52.0
5,48.0
6,37.0
7,40.0
8,40.0
9,40.0


A                        CALSIM                            \
B     S_SLUIS_SWPLEVEL1DV_s0001 S_SLUIS_SWPLEVEL1DV_s0002   
C                 STORAGE-LEVEL             STORAGE-LEVEL   
D                          1MON                      1MON   
E                        L2020A                    L2020A   
F                      PER-AVER                  PER-AVER   
Units                       TAF                       TAF   
0                      2.916667                  2.666667   

A                                                          \
B     S_SLUIS_SWPLEVEL1DV_s0003 S_SLUIS_SWPLEVEL1DV_s0004   
C                 STORAGE-LEVEL             STORAGE-LEVEL   
D                          1MON                      1MON   
E                        L2020A                    L2020A   
F                      PER-AVER                  PER-AVER   
Units                       TAF                       TAF   
0                          2.75                  5.583333   

A                                                          \
B     S_SLUIS_SWPLEVEL1DV_s0005 S_SLUIS_SWPLEVEL1DV_s0006   
C                 STORAGE-LEVEL             STORAGE-LEVEL   
D                          1MON                      1MON   
E                        L2020A                    L2020A   
F                      PER-AVER                  PER-AVER   
Units                       TAF                       TAF   
0                      4.333333                       4.0   

A                                                          \
B     S_SLUIS_SWPLEVEL1DV_s0007 S_SLUIS_SWPLEVEL1DV_s0008   
C                 STORAGE-LEVEL             STORAGE-LEVEL   
D                          1MON                      1MON   
E                        L2020A                    L2020A   
F                      PER-AVER                  PER-AVER   
Units                       TAF                       TAF   
0                      3.083333                  3.333333   

A                                                          \
B     S_SLUIS_SWPLEVEL1DV_s0009 S_SLUIS_SWPLEVEL1DV_s0010   
C                 STORAGE-LEVEL             STORAGE-LEVEL   
D                          1MON                      1MON   
E                        L2020A                    L2020A   
F                      PER-AVER                  PER-AVER   
Units                       TAF                       TAF   
0                      3.333333                  3.333333   

A                                                          \
B     S_SLUIS_SWPLEVEL1DV_s0011 S_SLUIS_SWPLEVEL1DV_s0012   
C                 STORAGE-LEVEL             STORAGE-LEVEL   
D                          1MON                      1MON   
E                        L2020A                    L2020A   
F                      PER-AVER                  PER-AVER   
Units                       TAF                       TAF   
0                           2.5                       2.5   

A                                                          \
B     S_SLUIS_SWPLEVEL1DV_s0013 S_SLUIS_SWPLEVEL1DV_s0014   
C                 STORAGE-LEVEL             STORAGE-LEVEL   
D                          1MON                      1MON   
E                        L2020A                    L2020A   
F                      PER-AVER                  PER-AVER   
Units                       TAF                       TAF   
0                      3.333333                  2.583333   

A                                                          
B     S_SLUIS_SWPLEVEL1DV_s0015 S_SLUIS_SWPLEVEL1DV_s0016  
C                 STORAGE-LEVEL             STORAGE-LEVEL  
D                          1MON                      1MON  
E                        L2020A                    L2020A  
F                      PER-AVER                  PER-AVER  
Units                       TAF                       TAF  
0                      2.166667                  2.583333

Threshold: Sept_S_SLUIS_SWP_frequency_hitting_level1


,Sept_Prob_S_SLUIS_SWP_dead
0,5.0
1,5.0
2,6.0
3,13.0
4,7.0
5,6.0
6,6.0
7,5.0
8,7.0
9,7.0


A                        CALSIM                            \
B     S_SLUIS_SWPLEVEL1DV_s0001 S_SLUIS_SWPLEVEL1DV_s0002   
C                 STORAGE-LEVEL             STORAGE-LEVEL   
D                          1MON                      1MON   
E                        L2020A                    L2020A   
F                      PER-AVER                  PER-AVER   
Units                       TAF                       TAF   
0                           5.0                       5.0   

A                                                          \
B     S_SLUIS_SWPLEVEL1DV_s0003 S_SLUIS_SWPLEVEL1DV_s0004   
C                 STORAGE-LEVEL             STORAGE-LEVEL   
D                          1MON                      1MON   
E                        L2020A                    L2020A   
F                      PER-AVER                  PER-AVER   
Units                       TAF                       TAF   
0                           6.0                      13.0   

A                                                          \
B     S_SLUIS_SWPLEVEL1DV_s0005 S_SLUIS_SWPLEVEL1DV_s0006   
C                 STORAGE-LEVEL             STORAGE-LEVEL   
D                          1MON                      1MON   
E                        L2020A                    L2020A   
F                      PER-AVER                  PER-AVER   
Units                       TAF                       TAF   
0                           7.0                       6.0   

A                                                          \
B     S_SLUIS_SWPLEVEL1DV_s0007 S_SLUIS_SWPLEVEL1DV_s0008   
C                 STORAGE-LEVEL             STORAGE-LEVEL   
D                          1MON                      1MON   
E                        L2020A                    L2020A   
F                      PER-AVER                  PER-AVER   
Units                       TAF                       TAF   
0                           6.0                       5.0   

A                                                          \
B     S_SLUIS_SWPLEVEL1DV_s0009 S_SLUIS_SWPLEVEL1DV_s0010   
C                 STORAGE-LEVEL             STORAGE-LEVEL   
D                          1MON                      1MON   
E                        L2020A                    L2020A   
F                      PER-AVER                  PER-AVER   
Units                       TAF                       TAF   
0                           7.0                       7.0   

A                                                          \
B     S_SLUIS_SWPLEVEL1DV_s0011 S_SLUIS_SWPLEVEL1DV_s0012   
C                 STORAGE-LEVEL             STORAGE-LEVEL   
D                          1MON                      1MON   
E                        L2020A                    L2020A   
F                      PER-AVER                  PER-AVER   
Units                       TAF                       TAF   
0                           5.0                       4.0   

A                                                          \
B     S_SLUIS_SWPLEVEL1DV_s0013 S_SLUIS_SWPLEVEL1DV_s0014   
C                 STORAGE-LEVEL             STORAGE-LEVEL   
D                          1MON                      1MON   
E                        L2020A                    L2020A   
F                      PER-AVER                  PER-AVER   
Units                       TAF                       TAF   
0                           6.0                       4.0   

A                                                          
B     S_SLUIS_SWPLEVEL1DV_s0015 S_SLUIS_SWPLEVEL1DV_s0016  
C                 STORAGE-LEVEL             STORAGE-LEVEL  
D                          1MON                      1MON  
E                        L2020A                    L2020A  
F                      PER-AVER                  PER-AVER  
Units                       TAF                       TAF  
0                           3.0                       4.0

In [16]:
# Other Metrics - not following excel file correctly

variables = [
    "S_RESTOT_NOD_",
    "S_SHSTA_", "S_OROVL_", "S_TRNTY_", "S_MELON_", "S_MLRTN_",
    "DEL_SWP_TOTAL_", "DEL_SWP_PMI_", "DEL_SWP_PAG_", "DEL_CVP_TOTAL_", "DEL_CVP_PAG_TOTAL_", "DEL_CVP_PSCEX_TOTAL_", "DEL_CVP_PRF_TOTAL_",
    "D_TOTAL_", "NDO_",
    "X2_PRV_KM_",
    "DEL_CVP_PSC_N_", "DEL_CVP_PAG_N_", "DEL_CVP_PAG_S_", "DEL_CVP_PRF_S_", "DEL_CVP_PMI_N_", "DEL_CVP_PMI_S_"
] 

In [17]:
metrics = {}

for var in variables:
    if var in ["C_SAC041_", "C_SJR070_", "C_SAC000_", "C_SJR070_", "C_DMC000_TD_", "C_CAA003_TD_", "NDO_"]:
        units = "CFS"
    elif var == "X2_PRV_KM_":
        units = "KM"
    else:
        units = "TAF"

    if var in ["S_RESTOT_NOD_"]: ## What's the difference between this and NOD_STORAGE?
        metrics[f"Apr_{var}mnth_avg"] = mnth_avg(df, dss_names, var, 4, units) 
        metrics[f"Sept_{var}mnth_avg"] = mnth_avg(df, dss_names, var, 9, units) 

    if var in ["S_SHSTA_", "S_OROVL_"]:
        metrics[f"Apr_{var}mnth_avg"] = mnth_avg(df, dss_names, var, 4, units) 

    ## I'M VERY CONFUSED ABOUT WHAT TO DO FOR THIS ONE. S_SHSTA, S_OROVL, S_TRNTY, S_MELON, S_MLRTN; dead pool set in *LEVEL1DV variables probabilities.
    ## What do I put as the percentile? I put 1. Not returning probability.
    if var in ["S_SHSTA_", "S_OROVL_", "S_TRNTY_", "S_MELON_", "S_MLRTN_"]:
        metrics[f"Sept_{var}mnth_percentile"] = mnth_percentile(df, dss_names, 1, var, 9, units)

    ## What do I put as the percentile? I put 1. Not returning probability.
    if var in ["S_SHSTA_"]:
        metrics[f"Apr_{var}mnth_percentile"] = mnth_percentile(df, dss_names, 1, var, 4, units)

    ## LONG-TERM CUMULATIVE FRACTION? What is this? S_SHSTA_ & S_SHSTALEVEL5DV

    if var in ["DEL_SWP_TOTAL_", "DEL_SWP_PMI_", "DEL_SWP_PAG_", "DEL_CVP_TOTAL_", "DEL_CVP_PAG_TOTAL_", "DEL_CVP_PSCEX_TOTAL_", "DEL_CVP_PRF_TOTAL_"]:
        metrics[f"{var}ann_avg"] = ann_avg(df, dss_names, var, units)
     
    ## ADD DEL_CVP_PAG_TOTAL +  DEL_CVP_PSCEX_TOTAL

    ## I DON'T SEE VARIABLE D_TOTAL
    ## LONG-TERM AVERAGE? What's the difference between that and average?
    if var in ["NDO_"]: # missing D_TOTAL
        metrics[f"{var}ann_avg"] = ann_avg(df, dss_names, var, units)

    if var in ["X2_PRV_KM_"]:
        metrics[f"Apr_{var}mnth_avg"] = mnth_avg(df, dss_names, var, 4, units) 
        metrics[f"Oct_{var}mnth_avg"] = mnth_avg(df, dss_names, var, 10, units) 
    ## NEED TO ADD WAY TO DO MULTI-MONTH 
        ## Sept-Nov
        ## March-May

    if var in ["DEL_SWP_PMI_", "DEL_CVP_PSC_N_", "DEL_CVP_PAG_N_", "DEL_CVP_PAG_S_", "DEL_CVP_PRF_S_", "DEL_CVP_PMI_N_", "DEL_CVP_PMI_S_"]:
        metrics[f"{var}ann_percentile"] = ann_percentile(df, dss_names, 0.9, var, units)
        
for var, metric_df in metrics.items():
    display(metric_df)

,Apr_Avg_S_RESTOT_NOD_TAF
s0001,9502.720622
s0002,9289.524626
s0003,8831.470114
s0004,8536.757009
s0005,8363.576594
s0006,8797.665441
s0007,8818.561157
s0008,8817.579762
s0009,8806.576365
s0010,8809.165562


,Sep_Avg_S_RESTOT_NOD_TAF
s0001,6800.449020
s0002,6570.594230
s0003,5655.789429
s0004,5292.440964
s0005,5134.175172
s0006,6152.457555
s0007,6193.778243
s0008,6194.621633
s0009,6187.818817
s0010,6188.742741


,Apr_Avg_S_SHSTA_TAF
s0001,4037.991051
s0002,3920.254867
s0003,3711.537404
s0004,3592.096895
s0005,3511.278046
s0006,4035.944251
s0007,4044.659226
s0008,4044.682619
s0009,4038.165765
s0010,4038.121174


,Sep_Percentile_S_SHSTA_TAF
s0001,3400.0
s0002,3400.0
s0003,3400.0
s0004,3400.0
s0005,3400.0
s0006,3400.0
s0007,3400.0
s0008,3400.0
s0009,3400.0
s0010,3400.0


,Apr_Percentile_S_SHSTA_TAF
s0001,4552.1
s0002,4552.1
s0003,4552.1
s0004,4552.1
s0005,4552.1
s0006,4552.1
s0007,4552.1
s0008,4552.1
s0009,4552.1
s0010,4552.1


...

,Percentile_DEL_CVP_PAG_N_TAF
s0001,34.847494
s0002,34.393217
s0003,33.918754
s0004,33.893795
s0005,33.723046
s0006,34.344864
s0007,34.336341
s0008,34.340126
s0009,34.336667
s0010,34.336660


,Percentile_DEL_CVP_PAG_S_TAF
s0001,143.389351
s0002,143.245424
s0003,138.804389
s0004,136.779033
s0005,135.285446
s0006,139.563284
s0007,142.365815
s0008,144.623838
s0009,144.630669
s0010,144.629032


,Percentile_DEL_CVP_PRF_S_TAF
s0001,23.905811
s0002,23.882565
s0003,23.922530
s0004,23.902585
s0005,23.890469
s0006,24.375671
s0007,24.375671
s0008,24.375671
s0009,24.375671
s0010,24.375671


,Percentile_DEL_CVP_PMI_N_TAF
s0001,1.298429
s0002,13.084345
s0003,1.298429
s0004,13.623956
s0005,1.381785
s0006,14.968287
s0007,1.381785
s0008,14.826032
s0009,1.381785
s0010,15.403063


,Percentile_DEL_CVP_PMI_S_TAF
s0001,13.722242
s0002,13.714244
s0003,13.691231
s0004,13.596215
s0005,13.498519
s0006,13.759796
s0007,13.768386
s0008,13.779971
s0009,13.779993
s0010,13.779988
